In [1]:
from sklearn import * 
import pandas as pd 
import numpy as np 

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.preprocessing import MinMaxScaler

In [2]:
def print_full(x):
    pd.set_option('display.max_rows' , len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
data = pd.read_csv('Kaggle.csv')

data.head()

,Id,Human Development Index HDI-2014,Gini coefficient 2005-2013,Adolescent birth rate 15-19 per 100k 20102015,Birth registration funder age 5 2005-2013,Carbon dioxide emissionsAverage annual growth,Carbon dioxide emissions per capita 2011 Tones,Change forest percentable 1900 to 2012,Change mobile usage 2009 2014,Consumer price index 2013,...,Renewable sources percentage of total 2012,Research and development expenditure 2005-2012,Secondary 2008-2014,Share of seats in parliament percentage held by womand 2014,Stock of immigrants percentage of population 2013,Taxes on income profit and capital gain 205 2013,Tertiary -2008-2014,Total tax revenue of GDP 2005-2013,Tuberculosis rate per thousands 2012,Under-five Mortality 2013 thousands
0,Norway,0.943877,26.83,7.834,100.0,0.778925,9.192879,11.914567,5.22,104.194175,...,47.752676,1.65474,111.06130,39.644970,13.772622,31.798391,74.10112,27.288097,0.14,2.8
1,Australia,0.934958,34.01,12.059,100.0,1.090351,16.519210,-4.561812,30.27,107.789440,...,4.632202,2.38562,135.53543,30.530973,27.711793,65.333748,86.33409,21.361426,0.19,4.0
2,Switzerland,0.929613,32.35,1.900,100.0,-1.101254,4.625230,8.567416,16.72,99.317229,...,49.659398,2.87046,96.30638,28.455285,28.906998,22.673299,55.56190,9.759124,0.22,4.2
3,Denmark,0.923328,26.88,5.101,100.0,-1.767733,7.248329,23.029974,1.83,106.057718,...,26.767245,2.98416,124.65927,37.988827,9.909512,39.677938,79.59763,33.395651,0.40,3.5
4,Netherlands,0.921794,28.87,6.165,100.0,-0.252734,10.064490,5.922602,-4.31,107.474154,...,6.671366,2.15676,129.91277,36.888889,11.724418,23.533104,77.34356,19.724059,0.17,4.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 66 columns):
 #   Column                                                                                Non-Null Count  Dtype  
---  ------                                                                                --------------  -----  
 0   Id                                                                                    188 non-null    object 
 1   Human Development Index HDI-2014                                                      188 non-null    float64
 2   Gini coefficient 2005-2013                                                            188 non-null    float64
 3   Adolescent birth rate 15-19 per 100k 20102015                                         188 non-null    float64
 4   Birth registration funder age 5 2005-2013                                             188 non-null    float64
 5   Carbon dioxide emissionsAverage annual growth                                        

In [4]:
ids = pd.DataFrame(data['Id'])
del data['Id']

min_max_scaler = MinMaxScaler()
original_dataset = min_max_scaler.fit_transform(data)

original_dataset

array([[1.00000000e+00, 4.90842491e-02, 3.53476481e-02, ...,
        4.64763791e-01, 9.79020979e-04, 4.83675937e-03],
       [9.85025710e-01, 2.24420024e-01, 5.60409851e-02, ...,
        3.63749932e-01, 1.32867133e-03, 1.20918984e-02],
       [9.76051726e-01, 1.83882784e-01, 6.28391748e-03, ...,
        1.66000924e-01, 1.53846154e-03, 1.33010883e-02],
       ...,
       [7.15967010e-02, 2.91330891e-01, 3.16708461e-01, ...,
        2.25768189e-01, 3.21678322e-02, 2.89600967e-01],
       [3.15078273e-03, 7.68742369e-01, 4.78601375e-01, ...,
        1.60820056e-01, 3.49650350e-01, 8.29504232e-01],
       [0.00000000e+00, 1.54822955e-01, 1.00000000e+00, ...,
        1.92908165e-01, 1.11888112e-01, 6.17896010e-01]])

In [5]:
ds = KMeans(n_clusters=3).fit(original_dataset)
clustered = pd.DataFrame(ids)
clustered['clusters'] = ds.labels_[:,np.newaxis]
clustered

,Id,clusters
0,Norway,2
1,Australia,2
2,Switzerland,2
3,Denmark,2
4,Netherlands,2
...,...,...
183,Burundi,0
184,Chad,0
185,Eritrea,0
186,Central African Republic,0


In [6]:
clustered['clusters'].value_counts()

1    78
2    55
0    55
Name: clusters, dtype: int64

In [7]:
#We can run PCA and extract one component and then cluster
pca = PCA(n_components=1)
pca.fit(original_dataset)

transf_dataset = pca.transform(original_dataset)
print(pca.explained_variance_ratio_)


Q = transf_dataset

clustered['PCA_1'] = Q
ds = KMeans(n_clusters=3).fit(Q)
clustered['Clustered_PCA'] = ds.labels_

[0.46397101]


In [8]:
siho = metrics.silhouette_score(data , ds.labels_)
print('Silhouette Coefficient : %0.3f'% metrics.silhouette_score(Q,clustered['Clustered_PCA']))

print('Silhouette Coefficient : %0.3f'% metrics.silhouette_score(original_dataset,clustered['Clustered_PCA']))

Silhouette Coefficient : 0.604
Silhouette Coefficient : 0.189


In [9]:
reporting_table = pd.concat((data , clustered), axis = 1)
grouped = reporting_table.groupby('Clustered_PCA').mean()
grouped

,Human Development Index HDI-2014,Gini coefficient 2005-2013,Adolescent birth rate 15-19 per 100k 20102015,Birth registration funder age 5 2005-2013,Carbon dioxide emissionsAverage annual growth,Carbon dioxide emissions per capita 2011 Tones,Change forest percentable 1900 to 2012,Change mobile usage 2009 2014,Consumer price index 2013,Domestic credit provided by financial sector 2013,...,Secondary 2008-2014,Share of seats in parliament percentage held by womand 2014,Stock of immigrants percentage of population 2013,Taxes on income profit and capital gain 205 2013,Tertiary -2008-2014,Total tax revenue of GDP 2005-2013,Tuberculosis rate per thousands 2012,Under-five Mortality 2013 thousands,clusters,PCA_1
Clustered_PCA,,,,,,,,,,,,,,,,,,,,,
0,0.698451,41.952480,47.915652,91.821256,3.254157,3.498879,0.380995,41.562029,119.226886,59.074095,...,84.310417,18.812204,5.088615,24.368006,30.506657,17.670885,7.790580,24.869565,0.956522,-0.041172
1,0.847652,32.441144,17.845308,99.636816,0.860438,9.203675,15.774132,17.779254,111.669963,112.515215,...,101.977079,23.581735,18.253035,22.291355,61.243323,17.342939,1.978408,7.028856,1.820896,-1.017473
2,0.484478,40.951154,91.752558,55.185897,1.203288,0.369870,-15.684736,130.407500,125.935747,28.126927,...,44.818659,19.341352,2.407655,21.746320,8.050087,15.463544,32.038462,81.782692,0.000000,1.365606


In [10]:
pd.set_option('display.max_rows' ,500)
pd.set_option('display.width',100)
grouped.transpose()

Clustered_PCA,0,1,2
Human Development Index HDI-2014,0.698451,0.847652,0.484478
Gini coefficient 2005-2013,41.952480,32.441144,40.951154
Adolescent birth rate 15-19 per 100k 20102015,47.915652,17.845308,91.752558
Birth registration funder age 5 2005-2013,91.821256,99.636816,55.185897
Carbon dioxide emissionsAverage annual growth,3.254157,0.860438,1.203288
Carbon dioxide emissions per capita 2011 Tones,3.498879,9.203675,0.369870
Change forest percentable 1900 to 2012,0.380995,15.774132,-15.684736
Change mobile usage 2009 2014,41.562029,17.779254,130.407500
Consumer price index 2013,119.226886,111.669963,125.935747
Domestic credit provided by financial sector 2013,59.074095,112.515215,28.126927
